# Arabic Sign Language - Landmark Training (Kaggle → TFLite)

Notebook steps:
- Authenticate Kaggle and download dataset outside the app
- Extract 21 hand landmarks per image with MediaPipe, normalize to 63 features
- Train a Dense classifier (63 → N classes)
- Convert to TensorFlow Lite and export `arabic_sign_lstm.tflite` and `labels.json`

Note: Keep labels order consistent across training and app.


In [ ]:
# Install dependencies
!pip -q install kaggle mediapipe opencv-python-headless==4.10.0.84 numpy pandas scikit-learn tensorflow==2.15.*

import os, json, zipfile, shutil, sys
from pathlib import Path
print("✅ Environment ready")


In [ ]:
# Kaggle authentication (upload your kaggle.json)
from google.colab import files

if not Path('/root/.kaggle/kaggle.json').exists():
    print('📥 Please upload kaggle.json (Profile → Account → Create New API Token)')
    uploaded = files.upload()  # choose kaggle.json
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)

print('✅ Kaggle API configured')


In [ ]:
# Download and unzip the dataset
DATASET_REF = 'muhammadalbrham/rgb-arabic-alphabets-sign-language-dataset'
DL_DIR = Path('dataset')
DL_DIR.mkdir(exist_ok=True)

!kaggle datasets download -d {DATASET_REF} -p {str(DL_DIR)}

# Unzip (force overwrite)
for z in DL_DIR.glob('*.zip'):
    print('Unzipping', z)
    !unzip -o {str(z)} -d {str(DL_DIR)}

print('✅ Dataset ready at', str(DL_DIR.resolve()))


In [ ]:
# Inspect dataset structure and build labels
import os

# Infer class directories (adjust if dataset layout differs)
class_dirs = []
for root, dirs, files in os.walk(DL_DIR):
    # consider leaf dirs with image files
    if any(f.lower().endswith(('.png', '.jpg', '.jpeg')) for f in files):
        class_dirs.append(Path(root))

# Heuristic: classes are the immediate subfolders of DL_DIR (adjust if needed)
classes = sorted({p.name for p in class_dirs if p.parent == DL_DIR})
print('Found classes:', classes)

# If heuristic fails, you can manually define classes like:
# classes = ["alef","baa","taa", ...]

with open('labels.json', 'w', encoding='utf-8') as f:
    json.dump(classes, f, ensure_ascii=False, indent=2)
print('✅ labels.json saved with', len(classes), 'classes')


In [ ]:
# Extract 21 hand landmarks (x,y,z) → 63 features per image
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp

with open('labels.json','r',encoding='utf-8') as f:
    labels = json.load(f)

hands = mp.solutions.hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.5
)

rows = []

def normalize_landmarks(xyz):
    # Match app logic closely: center at wrist (idx 0), scale by max L2 distance
    arr = np.array(xyz, dtype=np.float32)  # (21,3)
    origin = arr[0].copy()
    arr -= origin
    scale = np.max(np.linalg.norm(arr, axis=1)) + 1e-6
    arr /= scale
    return arr.reshape(-1)  # (63,)

for cls in labels:
    img_dir = DL_DIR/cls
    if not img_dir.exists():
        print('Skip (missing):', img_dir)
        continue
    files = [p for p in img_dir.iterdir() if p.suffix.lower() in {'.png','.jpg','.jpeg'}]
    for p in files:
        img = cv2.imread(str(p))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        res = hands.process(img)
        if not res.multi_hand_landmarks:
            continue
        lm = res.multi_hand_landmarks[0].landmark
        xyz = [(pt.x, pt.y, getattr(pt, 'z', 0.0)) for pt in lm]
        feats = normalize_landmarks(xyz)
        row = {f'f{i}': float(v) for i, v in enumerate(feats)}
        row['label'] = cls
        rows.append(row)

hands.close()

landmarks_df = pd.DataFrame(rows)
landmarks_df.to_csv('dataset_landmarks.csv', index=False)
print('✅ Saved landmarks CSV with shape:', landmarks_df.shape)


In [ ]:
# Train Dense model (63 → num_classes)
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

landmarks_df = pd.read_csv('dataset_landmarks.csv')
feature_cols = [c for c in landmarks_df.columns if c.startswith('f')]
X = landmarks_df[feature_cols].values.astype('float32')
y_text = landmarks_df['label'].values

le = LabelEncoder()
y_idx = le.fit_transform(y_text)
num_classes = len(le.classes_)
y = tf.keras.utils.to_categorical(y_idx, num_classes)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y_idx, random_state=42
)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(63,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

# Persist labels (order matters)
with open('labels.json', 'w', encoding='utf-8') as f:
    json.dump(list(le.classes_), f, ensure_ascii=False, indent=2)

print('✅ Training complete. Val acc:', float(history.history['val_accuracy'][-1]))


In [ ]:
# Convert to TFLite and export
import tensorflow as tf
from google.colab import files

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optional optimizations (uncomment to try size/speed trade-offs)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()
with open('arabic_sign_lstm.tflite', 'wb') as f:
    f.write(tflite_model)

print('✅ Saved arabic_sign_lstm.tflite and labels.json')

# Download to your computer
files.download('arabic_sign_lstm.tflite')
files.download('labels.json')
